In [1]:
from src.mapping import compute_area_scores

from src.alex_extractor import build_alexnet_design_matrices_with_dataloader

import itertools
import pandas as pd
import json
import matplotlib.pyplot as plt
import os

from itertools import product
from pathlib import Path
import pickle
from tqdm import tqdm

# Signals

## Model (Alexnet)

Producing the Alexnet activations for all the stimuli of the Allen dataset, for all the trained model and the untrained one

In [2]:
# Common params once
COMMON = dict(
    folder="../allen-data-clean/stimuli_images/",
    batch_size=16,
    shuffle=False,
    num_workers=4,
    device="cpu",
    amp=True,
    layers_keep=["conv1","conv2","conv3","conv4","conv5"],
    return_in_memory=False,  # keeps memmaps on disk (good for many models)
)

# Built-ins
weight_specs = [
    ("Random",   "random"),
    ("ImageNet", "imagenet"),
]

# # Add all local .pth/.pt in a directory (adjust path!)
ckpt_dir = Path("../models/weights/") 
for p in sorted(ckpt_dir.glob("*.pth")) + sorted(ckpt_dir.glob("*.pt")):
    label = p.stem
    weight_specs.append((label, str(p)))

diet_opts = ["diet", "diet-no-noise", "diet-no-blur", "no-diet", "random-diet", "nayebi-diet"] # diet options at inference time 

def make_save_dir(label: str, diet: bool) -> str:
    base = f"{label.lower().replace(' ', '-')}"
    return f"{base}-{diet}-act" 

index_models = {}
for (label, weights), diet in product(weight_specs, diet_opts):
    tag = f"{label}-{diet}"
    save_dir = make_save_dir(label, diet)
    index_models[tag] = build_alexnet_design_matrices_with_dataloader(
        **COMMON,
        weights=weights,  # "random" | "imagenet" | "/path/to.ckpt" (string) | {"file": "..."}
        diet=diet,
        save_dir=Path("activation") / save_dir,
    )


/home/giovanni/Desktop/mice-representation/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/giovanni/Desktop/mice-representation/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## Loading Neural Data

In [3]:
# # load the index containing the neural data
index_df = pd.read_csv('../allen-data-clean/PreprocData/index.csv')

# RSA

In [4]:
layer_scores_rsa = {}

# Wrap the loop with tqdm
for name, idx in tqdm(index_models.items(), desc="Computing area scores", total=len(index_models)):
    layers= compute_area_scores(idx, index_df, sim_metric="RSA")
    layer_scores_rsa[name] = layers

Computing area scores: 100%|██████████| 18/18 [20:32<00:00, 68.49s/it]


In [5]:
with open("rsa_results.pkl", "wb") as f:
    pickle.dump(layer_scores_rsa, f)